# 1. Introdução

Este trabalho (MVP - Minimum Viable Product) visa desenvolver uma solução real com base nos conhecimentos adquiridos na disciplina de Engenharia de Dados do curso de Pós-Graduação em Ciência de Dados da PUC Rio.
Aqui vamos construir um pipeline de dados em nuvem, contemplando as etapas de coleta, modelagem, carga e análise dos dados. 



O conjunto de dados utilizados para construir esse pipeline é o de _Temperature Change on Land_, disponível na plataforma FAOSTAT, mantida pela Organização das Nações Unidas para Alimentação e Agricultura (FAO).Os dados possuem informações sobre mudanças de temperatura em superfícies terrestres ao redor do mundo, com periodicidade anual por país entre 1961 e 2022.


# 2. Objetivo

O objetivo deste trabalho é armazenar e estruturar um conjunto de dados sobre mudanças de temperatura da superficie terrestre por país, utilizando um pipeline de dados em nuvem. Para isso, será realizada a modelagem do banco de dados de forma a garantir um bom desempenho, organização e facilidade na análise dos dados, utilizando conceitos de Data Warehouse e a arquitetura em camadas bronze, prata e ouro. Após o armazenamento e organização, serão conduzidas análises exploratórias com linguagem SQL, com o propósito de responder perguntas analíticas que permitam extrair insights relevantes sobre as variações de temperatura da superfície terrestre ao longo do tempo e entre diferentes países. 

As perguntas sobre o conjunto de dados a serem respondidas neste trabalho são as seguintes: 

  - As temperaturas da superfície está aumentando ou diminuindo?
  
  - Como se encontra o Brasil e neste cenário? Aquecendo ou esfriando?
  
  - Há alguma região do planeta indo contra a tendência global?

  - Qual é tendência das temperaturas nos países considerados potências econômicas regionais e globais?

# 3. Plataforma de Dados em Nuvem

A plataforma utilizada foi o Databricks Community Edition. Entre as principais vantagens do Community Edition estão a compatibilidade nativa com Apache Spark, o suporte a múltiplas linguagens de programação (como Python e SQL) e a interface intuitiva baseada em notebooks. Apesar de possuir algumas restrições de performance e armazenamento, e recursos limitados em comparação à versão paga (Databricks Premium), a gratuidade da versão Community torna a plataforma ideal para fins educacionais e a realização de projetos de pequeno porte.

# 4. Dados Utilizados

## 4.1 Busca e Coleta de Dados

Com o objetivo de identificar fontes confiáveis e atualizadas sobre indicadores de mudanças climáticas globais, em especial relacionados à variação de temperatura ao longo dos anos, foi realizada uma uma pesquisa exploratória em diversas bases de dados públicas e gratuitas disponíveis na internet .

Após essa investigação, oconjunto de dados selecionados  foi encontrado no trabalho 🔗[Climate Change Indicators](https://www.kaggle.com/datasets/tarunrm09/climate-change-indicators?resource=download) no site da Kaggle. Esses dados são provenientes do portal da FAOSTAT, mantido pela Organização das Nações Unidas para a Alimentação e a Agricultura (FAO). A FAOSTAT é uma plataforma de referência internacional que disponibiliza uma ampla gama de estatísticas relacionadas à agricultura, meio ambiente e sustentabilidade, de forma acessível e gratuita.






## 4.2 Explicando os Dados

Os dados de mudança de temperatura sobre a terra (_Temperature change on land_) da FAOSTAT possuem informações referentes à variação média da temperatura da superfície terrestre, organizadas por país, com atualizações anuais. A divulgação usada aqui cobre o período de 1961 a 2024,e as variações de temperatura são calculados com base em uma climatologia de referência estabelecida entre 1951 e 1980. Os dados incluem:

  - Anomalias anuais de temperatura, ou seja, variações em relação à média histórica do período de base.

  - Dados referentes a 188 países e 38 territórios, de acordo com o padrão estatístico da FAOSTAT.

A base de dados é construída a partir do GISTEMP – a série histórica de mudanças de temperatura da superfície terrestre disponibilizada pela NASA-GISS (Goddard Institute for Space Studies). A coleta de dados é realizada por meio de aproximadamente 26.000 estações meteorológicas em todo o mundo, por meio da rede NOAA-GHCN v4 (Global Historical Climatology Network), combinando medições reais com modelagem espacial.


> Catáogo de Dados

Com o objetivo de estruturar, documentar e facilitar a compreensão e o uso adequado dessas informações, foi desenvolvido um catálogo de dados detalhado para este trabalho. O catálogo apresenta a descrição técnica dos atributos presentes no conjunto de dados, abrangendo o tipo de dado, domínio de valores válidos, regras de validação, além da caracterização das entidades e relacionamentos, com base no modelo dimensional de esquema estrela utilizado na camada prata da arquitetura medalhão. A construção do catálogo permite garantir maior governança, qualidade e rastreabilidade dos dados, além de ser essencial para análises futuras, integração com outros sistemas e replicação do processo de modelagem.

Para mais informações sobre os dados utilizados, suas estruturas e definições técnicas, consulte o **catálogo de dados** deste trabalho, onde estão documentados todos os elementos relevantes do conjunto de dados.

## 4.3 Modelagem de Dados

A modelagem é uma etapa fundamental para garantir a eficiência, qualidade e escalabilidade de soluções analíticas em ambientes de dados. No caso de indicadores globais de mudança de temperatura na superfície terrestre, que abrangem registros históricos por país, será feita a aplicação conjunta do esquema em estrela (_star schema_) e da arquitetura em camadas ou medalhão (_medallion architecture_). A combinação entre o esquema estrela e a arquitetura medalhão representa uma abordagem robusta para a modelagem de dados, pois enquanto o esquema estrela viabiliza análises ágeis e estruturadas a partir de uma semântica bem definida, a arquitetura medalhão garante governança, versionamento e qualidade dos dados em cada etapa do pipeline.

### 4.3.1 Esquema Estrela

O esquema em estrela é um modelo de dados dimensional amplamente adotado em soluções de Business Intelligence e Data Warehousing. Ele se baseia na centralização de uma tabela fato, com os dados quantitativos e métricas, e em tabelas dimensão, com os atributos descritivos das variáveis categóricas.

No contexto dos dados aqui utilizados, a tabela fato (fato_temperatura) armazenará os valores de variação de temperatura por país e ano. Já as tabelas dimensão descreverão o país (dim_pais), o tempo (dim_tempo) e o tipo de indicador (dim_indicador).

A estrutura de separação em dimensões distintas proporciona diversas vantagens, entre elas:
  - Compatibilidade com ferramentas de visualização e construção de dashboards (ex: Power BI, Tableau, etc.);
  
  - Redução da redundância de dados, uma vez que os atributos descritivos são separados em dimensões reutilizáveis;
  
  - Alta performance em agregações (por país, tempo, etc).

### 4.3.2 Arquitetura Medalhão

A arquitetura medalhão organiza os dados em camadas sequenciais, promovendo governança, rastreabilidade, consistência e facilidade de transformação. As camadas que serão utilizadas neste trabalho são:

🥉 **Camada Bronze – Dados Brutos**

O objetivo desta camada é preservar os dados exatamente como foram extraídos da fonte, sem qualquer transformação.

🥈 **Camada Prata – Dados Limpos e Estruturados**

Aqui o objetivo é realizar a limpeza, normalização e padronização dos dados.

🥇 Camada Ouro – Dados Analíticos

Na camada ouro, o objetivo é criar uma estrutura analítica otimizada para consumo por ferramentas de BI e dashboards.

# 5. Análise

## 5.1 Carga dos dados no DataBricks

Para iniciar o processo de análise de dados, o arquivo CSV contendo as informações sobre variações de temperatura foi importado para o ambiente de nuvem do Databricks Community. A importação foi realizada por meio do DBFS (Databricks File System), um sistema de arquivos distribuído que permite o armazenamento e acesso eficiente de arquivos dentro da plataforma.

O upload foi feito diretamente pela interface web do Databricks, garantindo que o arquivo ficasse disponível em um caminho acessível para leitura e manipulação dentro dos notebooks. A partir desse arquivo, foi possível criar DataFrames em PySpark e dar início ao pipeline de tratamento, modelagem e análise dos dados.

### Camada Bronze – Ingestão dos dados

O passo seguinte foi a criação da camada Bronze, a primeira da arquitetura em camadas do modelo medalhão. Nessa etapa, os dados brutos foram carregados a partir do arquivo CSV importado via DBFS e salvos em formato Delta. Essa camada tem como principal objetivo armazenar os dados em um único DataFrame, sem alterações, preservando a integridade da fonte original e servindo como base para as etapas seguintes de limpeza e transformação dos dados.




In [0]:
# Criar o schema Bronze
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")

# Leitura do CSV
df_bronze = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("dbfs:/FileStore/tables/climate_change_indicators-9.csv")
)

# Salvando como Delta Table no schema bronze
df_bronze.write.format("delta").mode("overwrite").saveAsTable("bronze.climate_change_raw")

# Visualização das primeiras 5 linhas da tabela bronze
display(spark.table("bronze.climate_change_raw").limit(5))

ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,F1961,F1962,F1963,F1964,F1965,F1966,F1967,F1968,F1969,F1970,F1971,F1972,F1973,F1974,F1975,F1976,F1977,F1978,F1979,F1980,F1981,F1982,F1983,F1984,F1985,F1986,F1987,F1988,F1989,F1990,F1991,F1992,F1993,F1994,F1995,F1996,F1997,F1998,F1999,F2000,F2001,F2002,F2003,F2004,F2005,F2006,F2007,F2008,F2009,F2010,F2011,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
1,"Afghanistan, Islamic Rep. of",AF,AFG,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",-0.113,-0.164,0.847,-0.764,-0.244,0.226,-0.371,-0.423,-0.539,0.813,0.619,-1.124,0.232,-0.489,-0.445,-0.286,0.513,0.129,0.361,0.6,0.483,-0.346,0.164,0.145,0.283,-0.141,0.391,0.919,-0.205,0.73,-0.168,-0.294,0.22,0.43,0.359,-0.116,0.471,0.675,1.198,0.993,1.311,1.365,0.587,1.373,0.401,1.72,0.675,0.704,0.895,1.613,1.397,0.223,1.281,0.456,1.093,1.555,1.54,1.544,0.91,0.498,1.327,2.012
2,Albania,AL,ALB,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",0.627,0.326,0.075,-0.166,-0.388,0.559,-0.074,0.081,-0.013,-0.106,-0.195,-0.069,-0.288,-0.139,-0.211,-0.683,0.545,-0.814,0.203,-0.414,-0.351,0.173,-0.128,-0.27,-0.103,0.569,-0.106,0.37,-0.066,0.795,-0.269,0.106,0.076,1.33,-0.172,-0.038,0.075,0.795,0.67,1.065,1.532,0.492,0.97,0.444,0.189,0.345,1.316,0.978,0.91,1.191,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498,1.536,1.518
3,Algeria,DZ,DZA,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",0.164,0.114,0.077,0.25,-0.1,0.433,-0.026,-0.067,0.291,0.116,-0.385,-0.348,-0.015,-0.503,-0.539,-0.782,0.504,0.012,0.654,0.232,0.215,0.399,0.56,-0.004,0.508,0.296,0.975,1.304,0.386,1.266,0.031,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,0.82,1.856,1.258,1.585,0.988,1.264,1.395,1.22,1.185,0.945,2.265,1.398,1.147,1.192,1.69,1.121,1.757,1.512,1.21,1.115,1.926,2.33,1.688
4,American Samoa,AS,ASM,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",0.079,-0.042,0.169,-0.14,-0.562,0.181,-0.368,-0.187,0.132,-0.047,-0.477,-0.067,0.33,-0.308,-0.118,-0.177,0.156,0.092,0.341,0.35,0.179,0.28,0.313,0.277,0.256,0.394,0.354,0.509,0.143,0.497,0.641,0.344,-0.069,0.189,0.755,0.784,null,null,0.242,0.626,0.904,1.152,0.716,0.191,0.801,0.403,1.032,0.67,null,1.311,0.854,0.924,1.257,1.17,1.009,1.539,1.435,1.189,1.539,1.43,1.268,1.256
5,"Andorra, Principality of",AD,AND,"Temperatu

In [0]:
# Leitura da camada bronze
df_bronze = spark.table("bronze.climate_change_raw")

# Verificar schema atual
df_bronze.printSchema()

root
 |-- ObjectId: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- ISO2: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- CTS_Code: string (nullable = true)
 |-- CTS_Name: string (nullable = true)
 |-- CTS_Full_Descriptor: string (nullable = true)
 |-- F1961: double (nullable = true)
 |-- F1962: double (nullable = true)
 |-- F1963: double (nullable = true)
 |-- F1964: double (nullable = true)
 |-- F1965: double (nullable = true)
 |-- F1966: double (nullable = true)
 |-- F1967: double (nullable = true)
 |-- F1968: double (nullable = true)
 |-- F1969: double (nullable = true)
 |-- F1970: double (nullable = true)
 |-- F1971: double (nullable = true)
 |-- F1972: double (nullable = true)
 |-- F1973: double (nullable = true)
 |-- F1974: double (nullable = true)
 |-- F1975: double (nullable = true)
 |-- F1976: double (nullable = true)
 |-- 

###  Camada Prata – Transformação

A partir deste ponto, foram realizados ajustes nos dados visando a organização e eficiência das análises. Na camada prata serão execudatas as seguintes atividades:

  - Conversão de tipos de dados das colunas de ano de double para float.
  
  - Transformação de colunas ano (F1961, F1962, ..., F2024) para formato longitudinal (unpivot), de forma que cada linha represente um único valor por ano.
 
  - Remoção de colunas não utilizadas ou redundantes, como ISO2.
  
  - Padronização do nome de países e elementos para garantir consistência.



In [0]:
from pyspark.sql.functions import col, expr

# Criação do schema Prata
spark.sql("CREATE SCHEMA IF NOT EXISTS silver")



Out[3]: DataFrame[]

A coluna ISO2 foi removida por conter informação redundante com a coluna ISO3, que também representa a sigla dos países. Optou-se por manter apenas a ISO3, por ser a forma mais comum e padronizada de identificação internacional de países, com três letras, o que facilita a leitura e compreensão.

Além disso, os atributos referentes aos anos estavam originalmente no formato double e foram convertidos para o tipo float, que é suficiente para representar valores com pequena variação decimal e possuem a precisão necessária para as análises propostas neste trabalho.

In [0]:
# Remoção da coluna ISO2
df_silver = df_bronze.drop("ISO2")


# Conversão dos anos F1961–F2022 para float
anos = [f"F{ano}" for ano in range(1961, 2023)]

for col_ano in anos:
    df_silver = df_silver.withColumn(col_ano, col(col_ano).cast("float"))

# Salvando como Delta Table no schema silver
df_silver.write.format("delta").mode("overwrite").saveAsTable("silver.climate_change_edited")

# Leitura da camada silver
df_silver = spark.table("silver.climate_change_edited")

# Verificar schema atual
df_silver.printSchema()

# Visualização das primeiras 5 linhas da tabela silver
display(spark.table("silver.climate_change_edited").limit(5))



root
 |-- ObjectId: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- CTS_Code: string (nullable = true)
 |-- CTS_Name: string (nullable = true)
 |-- CTS_Full_Descriptor: string (nullable = true)
 |-- F1961: float (nullable = true)
 |-- F1962: float (nullable = true)
 |-- F1963: float (nullable = true)
 |-- F1964: float (nullable = true)
 |-- F1965: float (nullable = true)
 |-- F1966: float (nullable = true)
 |-- F1967: float (nullable = true)
 |-- F1968: float (nullable = true)
 |-- F1969: float (nullable = true)
 |-- F1970: float (nullable = true)
 |-- F1971: float (nullable = true)
 |-- F1972: float (nullable = true)
 |-- F1973: float (nullable = true)
 |-- F1974: float (nullable = true)
 |-- F1975: float (nullable = true)
 |-- F1976: float (nullable = true)
 |-- F1977: float (nullable = true)
 |-- F1978: float (nu

ObjectId,Country,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,F1961,F1962,F1963,F1964,F1965,F1966,F1967,F1968,F1969,F1970,F1971,F1972,F1973,F1974,F1975,F1976,F1977,F1978,F1979,F1980,F1981,F1982,F1983,F1984,F1985,F1986,F1987,F1988,F1989,F1990,F1991,F1992,F1993,F1994,F1995,F1996,F1997,F1998,F1999,F2000,F2001,F2002,F2003,F2004,F2005,F2006,F2007,F2008,F2009,F2010,F2011,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
1,"Afghanistan, Islamic Rep. of",AFG,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",-0.113,-0.164,0.847,-0.764,-0.244,0.226,-0.371,-0.423,-0.539,0.813,0.619,-1.124,0.232,-0.489,-0.445,-0.286,0.513,0.129,0.361,0.6,0.483,-0.346,0.164,0.145,0.283,-0.141,0.391,0.919,-0.205,0.73,-0.168,-0.294,0.22,0.43,0.359,-0.116,0.471,0.675,1.198,0.993,1.311,1.365,0.587,1.373,0.401,1.72,0.675,0.704,0.895,1.613,1.397,0.223,1.281,0.456,1.093,1.555,1.54,1.544,0.91,0.498,1.327,2.012
2,Albania,ALB,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",0.627,0.326,0.075,-0.166,-0.388,0.559,-0.074,0.081,-0.013,-0.106,-0.195,-0.069,-0.288,-0.139,-0.211,-0.683,0.545,-0.814,0.203,-0.414,-0.351,0.173,-0.128,-0.27,-0.103,0.569,-0.106,0.37,-0.066,0.795,-0.269,0.106,0.076,1.33,-0.172,-0.038,0.075,0.795,0.67,1.065,1.532,0.492,0.97,0.444,0.189,0.345,1.316,0.978,0.91,1.191,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498,1.536,1.518
3,Algeria,DZA,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",0.164,0.114,0.077,0.25,-0.1,0.433,-0.026,-0.067,0.291,0.116,-0.385,-0.348,-0.015,-0.503,-0.539,-0.782,0.504,0.012,0.654,0.232,0.215,0.399,0.56,-0.004,0.508,0.296,0.975,1.304,0.386,1.266,0.031,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,0.82,1.856,1.258,1.585,0.988,1.264,1.395,1.22,1.185,0.945,2.265,1.398,1.147,1.192,1.69,1.121,1.757,1.512,1.21,1.115,1.926,2.33,1.688
4,American Samoa,ASM,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",0.079,-0.042,0.169,-0.14,-0.562,0.181,-0.368,-0.187,0.132,-0.047,-0.477,-0.067,0.33,-0.308,-0.118,-0.177,0.156,0.092,0.341,0.35,0.179,0.28,0.313,0.277,0.256,0.394,0.354,0.509,0.143,0.497,0.641,0.344,-0.069,0.189,0.755,0.784,null,null,0.242,0.626,0.904,1.152,0.716,0.191,0.801,0.403,1.032,0.67,null,1.311,0.854,0.924,1.257,1.17,1.009,1.539,1.435,1.189,1.539,1.43,1.268,1.256
5,"Andorra, Principality of",AND,"Temperature change with respe

Os próximos passos consistem em:

  - Aplicar a operação de unpivot para transformar as colunas referentes aos anos em linhas, reorganizando os dados em um formato mais adequado para análises temporais;
  
  - Remover as linhas com valores nulos antes da criação da tabela fato, uma vez que essas informações são desconhecidas e, portanto, não serão consideradas nas análises sobre o comportamento da temperatura na superfície terrestre.

In [0]:
 # Unpivot dos anos
stack_expr = ", ".join([f"'{ano}', `{f'F{ano}'}`" for ano in range(1961, 2023)])

df_unpivot = df_silver.selectExpr(
    "Country", "ISO3", "Indicator", "Unit", "Source",
    "CTS_Code", "CTS_Name", "CTS_Full_Descriptor",
    f"stack({len(anos)}, {stack_expr}) as (ano, valor_temperatura)"
).withColumn("ano", col("ano").cast("int"))

# Remover linhas com valores nulos
df_unpivot = df_unpivot.dropna()

# Escreve tabela fato
df_unpivot.write.format("delta").mode("overwrite").saveAsTable("silver.fato_temperatura_prata")

# Visualização das primeiras 5 linhas da tabela silver
display(spark.table("silver.fato_temperatura_prata").limit(5))

Country,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,ano,valor_temperatura
"Afghanistan, Islamic Rep. of",AFG,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",1961,-0.113
"Afghanistan, Islamic Rep. of",AFG,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",1962,-0.164
"Afghanistan, Islamic Rep. of",AFG,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",1963,0.847
"Afghanistan, Islamic Rep. of",AFG,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",1964,-0.764
"Afghanistan, Islamic Rep. of",AFG,"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change",1965,-0.244


### Camada Ouro – Visualizações e Análises

Nesta camada será feita a criação da tabelas finais fato fato_temperatura e das tabelas dimensão:dim_pais, dim_tempo, dim_elemento. Essas versões serão usadas para as análises em SQL feitas com intuito de responder as perguntas feitas no objetivo deste trabalho.


In [0]:
# Criação do schema Ouro
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

Out[16]: DataFrame[]

> Criacao da tabela Fato 

Criaçõ da tabela fato final, composta pelas colunas listadas abaixo:

- codigo_iso3 - Código ISO3 do pais. Identificador da dimensão país/território
- codigo_CTS - Identificador da dimensão elemento climático
- ano - Identificador da dimensão tempo
- valor_temperatura - Anomalia de temperatura em graus Celsius (diferença da média 1951–1980)


In [0]:
# Escreve tabela fato
df_fato_gold = df_unpivot \
    .select("ISO3", "CTS_Code", "ano", "valor_temperatura") \
    .distinct() \
    .withColumnRenamed("CTS_Code", "codigo_CTS") \
    .withColumnRenamed("ISO3", "codigo_iso3")

df_fato_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.fato_temperatura")

# Visualização das primeiras 5 linhas da tabela silver
display(spark.table("gold.fato_temperatura").limit(5))

codigo_iso3,codigo_CTS,ano,valor_temperatura
AFG,ECCS,1978,0.129
BGD,ECCS,1981,-0.17
BRB,ECCS,1985,-0.263
BLR,ECCS,2010,1.461
BEN,ECCS,1972,0.08


> Tabela Dimensao Pais

Criação da tabela de dimensão País com as seguintes colunas:

- codigo_iso3 - Código ISO3 do pais
- nome_pais - Nome do país ou território


In [0]:
# Tabela dimensão país
df_dim_pais_gold = df_unpivot.select("Country", "ISO3").distinct() \
    .withColumnRenamed("Country", "nome_pais") \
    .withColumnRenamed("ISO3", "codigo_iso3") \
    .write.format("delta").mode("overwrite").saveAsTable("gold.dim_pais")

# Visualização das primeiras 5 linhas da tabela silver
display(spark.table("gold.dim_pais").limit(5))

nome_pais,codigo_iso3
Faroe Islands,FRO
Bhutan,BTN
Zambia,ZMB
Indonesia,IDN
Iraq,IRQ


> Tabela Dimensao Tempo

Criação da tabela de dimensão Tempo com as seguintes colunas:

- ano -	Ano da observação
- decada - Decada da observação


In [0]:
#  Tabela dimensão tempo
df_dim_tempo_gold = df_unpivot.select("ano").distinct() \
    .withColumn("decada", (col("ano") / 10).cast("int") * 10) \
    .write.format("delta").mode("overwrite").saveAsTable("gold.dim_tempo")

# Visualização das primeiras 5 linhas da tabela silver Dimensao tempo
display(spark.table("gold.dim_tempo").limit(5))

ano,decada
1990,1990
1975,1970
1977,1970
2003,2000
2007,2000


> Tabela Dimensao Indicador

Criação da tabela de dimensão Tempo com as seguintes colunas:

- nome_indicador - Nome do elemento (climático)
- unidade - Unidade de medição 
- fonte - Fonte dos dados
- codigo_CTS - Código do indicador climático
- nome_CTS - Nome do indicador climático
- descricao_CTS - Descrição completa do indicador climático


In [0]:
# Tabela dimensão indicador
df_dim_indicador_gold = df_unpivot .select("Indicator", "Unit", "Source", "CTS_Code", "CTS_Name", "CTS_Full_Descriptor").distinct() \
    .withColumnRenamed("Indicator", "nome_indicador") \
    .withColumnRenamed("Unit", "unidade") \
    .withColumnRenamed("Source", "fonte") \
    .withColumnRenamed("CTS_Code", "codigo_CTS") \
    .withColumnRenamed("CTS_Name", "nome_CTS") \
    .withColumnRenamed("CTS_Full_Descriptor", "descricao_CTS") \
    .write.format("delta").mode("overwrite").saveAsTable("gold.dim_indicador")

# Visualização das primeiras 5 linhas da tabela silver Dimensao Indicador
display(spark.table("gold.dim_indicador").limit(5))

nome_indicador,unidade,fonte,codigo_CTS,nome_CTS,descricao_CTS
"Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980",Degree Celsius,"Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.",ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicators, Surface Temperature Change"


# Análises em SQL

Consultas em SQL (Structured Query Language) são amplamente utilizadas em bancos de dados por permitirem extrair, filtrar, agrupar e analisar informações de forma estruturada e eficiente. Essa linguagem é essencial para transformar dados armazenados em insumos valiosos para tomada de decisão. Neste trabalho, utilizaremos consultas em SQL para responder às questões propostas no objetivo, explorando o comportamento das mudanças de temperatura ao longo do tempo e entre diferentes países.

Compare as temperaturas entre as cinco maiores economias da América do Sul no período entre 1961 e 2022.

Quantos países tiveram  valores de temperatura maiores que 1.5 em 1961? Caso haja, quais foram esses países?

In [0]:
%sql
SELECT 
    COUNT(DISTINCT f.ISO3) AS quantidade_paises
FROM 
    gold.fato_temperatura f
WHERE 
    f.ano = 1961
    AND f.valor_temperatura > 1.5;


quantidade_paises
2


In [0]:
%sql
 SELECT 
    dp.nome_pais,
    ROUND(f.valor_temperatura, 3) AS Valor_Temperatura
  
FROM 
     gold.fato_temperatura f
JOIN gold.dim_pais dp 
     ON f.codigo_iso3 = dp.codigo_iso3
WHERE 
    f.ano = 1961
    AND f.valor_temperatura > 1.5;

nome_pais,Valor_Temperatura
Finland,1.892
Sweden,1.563


No ano de 1961 período de coleta de dados apenas dois países, Finlância e Suécia, apresentaram aqueimentos maiores que 1.5 graus Celcius em sua superfície.

Quantos países tiveram  valores de temperatura maiores que 1.5 em 2022?

In [0]:
%sql
SELECT 
    COUNT(DISTINCT f.ISO3) AS quantidade_paises
FROM 
    gold.fato_temperatura f
WHERE 
    f.ano = 2022
    AND f.valor_temperatura > 1.5;

quantidade_paises
76


Com o passar dos anos podemos observar que a quantidade de países com aquecimentos mais intensos aumentou significativamente.

 Considerando todo o período de coleta de dados (1961 - 20022), qual é a temperatura média global da superficie por ano?

In [0]:
%sql
--
SELECT DISTINCT ano, ROUND(AVG(valor_temperatura), 2) AS media_global
FROM gold.fato_temperatura
GROUP BY ano
ORDER BY ano;

ano,media_global
1961,0.16
1962,-0.01
1963,-0.01
1964,-0.07
1965,-0.25
1966,0.11
1967,-0.11
1968,-0.2
1969,0.16
1970,0.09


Databricks visualization. Run in Databricks to view.

Podemos notar que há uma tendência de aumento no valor das anomalias das temperaturas das supefícies, intensificado apartir da década de 90.

Qual é a temperatura média global por decada?

In [0]:
%sql
--
SELECT t.decada, ROUND(AVG(f.valor_temperatura), 2) AS media_decada
FROM gold.fato_temperatura f
JOIN gold.dim_tempo t ON f.ano = t.ano
GROUP BY t.decada
ORDER BY t.decada;

decada,media_decada
1960,-0.02
1970,0.01
1980,0.24
1990,0.52
2000,0.85
2010,1.16
2020,1.43


Databricks visualization. Run in Databricks to view.

Podemos notar que, a partir da década de 80, as médias nas temperaturas das superfícies vêm aumentanto em torno de 0.3 C por década.

**A temperatura média da superfície no Brasil no período entre 1961 e 2022 também aumentou?**

In [0]:
%sql
--
SELECT DISTINCT ano, ROUND(AVG(valor_temperatura), 2) AS media_Brasil
FROM gold.fato_temperatura
WHERE ISO3 IN ("BRA")
GROUP BY ano
ORDER BY ano;

ano,media_Brasil
1961,0.17
1962,-0.18
1963,0.16
1964,-0.21
1965,-0.08
1966,0.04
1967,0.07
1968,-0.41
1969,0.33
1970,0.13


Databricks visualization. Run in Databricks to view.

Sim, podemos notar que a tendência das temperaturas no Brasil é a mesma que a tendência mundial.

Algum país apresentou resfriamento de sua superfície em 2022?

In [0]:
%sql
SELECT 
    dp.nome_pais,
    f.ano,
    ROUND(f.valor_temperatura, 3) AS Valor_Temperatura
  
FROM 
     gold.fato_temperatura f
JOIN gold.dim_pais dp 
     ON f.codigo_iso3 = dp.codigo_iso3
WHERE 
    f.ano = 2022
    AND f.valor_temperatura < 0;

nome_pais,ano,Valor_Temperatura
Botswana,2022,-1.3
Zimbabwe,2022,-0.49


Qual é a tendência da temperatura média da superfície em Botwana no período entre 1961 e 2022?

In [0]:
%sql
SELECT DISTINCT ano, ROUND(AVG(valor_temperatura), 2) AS media_Botwana
FROM gold.fato_temperatura
WHERE ISO3 IN ("BWA")
GROUP BY ano
ORDER BY ano;

ano,media_Botwana
1961,0.15
1962,0.26
1963,-0.47
1964,-0.06
1965,0.1
1966,0.44
1967,-0.46
1968,-0.19
1969,0.2
1970,0.6


Databricks visualization. Run in Databricks to view.

Botwana, diferentemente da temperatura mundial, apresenta uma tendência de estabilidade e uma queda acentuada após 2019.

Compare as temperaturas nas três maiores economias da América do Sul.

In [0]:
%sql
--
SELECT
    t.ano,
    p.nome_pais,
    ROUND(AVG(f.valor_temperatura), 3) AS temperatura_media_America_do_Sul
FROM
    gold.fato_temperatura f
JOIN gold.dim_pais p
    ON f.ISO3 = p.codigo_iso3
JOIN gold.dim_tempo t
    ON f.ano = t.ano
WHERE
    p.codigo_iso3 IN ('BRA', 'ARG', 'COL')
GROUP BY
    t.ano, p.nome_pais
ORDER BY
    t.ano, p.nome_pais;

ano,nome_pais,temperatura_media_America_do_Sul
1961,Argentina,0.122
1961,Brazil,0.167
1961,Colombia,0.09
1962,Argentina,-0.046
1962,Brazil,-0.184
1962,Colombia,-0.133
1963,Argentina,0.162
1963,Brazil,0.158
1963,Colombia,-0.058
1964,Argentina,-0.343


Databricks visualization. Run in Databricks to view.

Podemos observar que na América do Sul, Brasil, Argentia e Colômbia seguem a tendência mundial, com as médias brasileira ligeiramente acima da argentina e da comlombiana a partir da década de 90.

Faça uma comparação entre as temperaturas das superfícies do Brasil, China e Estados Unidos ao longo das décadas 60 até 20.

In [0]:
%sql
--
SELECT
    t.decada,
    p.nome_pais,
    ROUND(AVG(f.valor_temperatura), 3) AS temperatura_media
FROM
    gold.fato_temperatura f
JOIN gold.dim_pais p
    ON f.ISO3 = p.codigo_iso3
JOIN gold.dim_tempo t
    ON f.ano = t.ano
WHERE
    p.codigo_iso3 IN ('BRA', 'USA', 'CHN')
GROUP BY
    t.decada, p.nome_pais
ORDER BY
    t.decada, p.nome_pais;

decada,nome_pais,temperatura_media
1960,Brazil,-0.012
1960,"China, P.R.: Mainland",-0.029
1960,United States,-0.081
1970,Brazil,0.035
1970,"China, P.R.: Mainland",0.052
1970,United States,-0.102
1980,Brazil,0.324
1980,"China, P.R.: Mainland",0.191
1980,United States,0.351
1990,Brazil,0.614


Databricks visualization. Run in Databricks to view.

Todos apresentam tendência de aumento nas temperaturas das superfícies a partir da década de 80. Vale ressaltar que a China apresentou o maior aumento nas média das temperaturas entre as décadas de 10 e o ínicio da década de 20, com Brasil e estados unidos se mantendo praticamente estávveis.

### Resumo Geral das Análises em SQL

As consultas SQL executadas sobre os dados tratados na camada Gold permitiram extrair insights relevantes sobre o comportamento da temperatura média da superfície terrestre ao longo do tempo e entre diferentes países.

Observou-se uma tendência geral de aumento nas anomalias das temperaturas das superfícies, intensificada a partir da década de 1990.No entanto, desde a década de 1980, as temperaturas médias vêm subindo de forma consistente, com um acréscimo aproximado de 0,3 °C por década. Foi identificado que apenas dois países apresentaram valores médios de temperatura inferiores a 0 °C, indicando um resfriamento na superfície, no ano de 2022, o que sugere que é necessário averiguar o que acontece localmente nessa região.

Além disso, 76 países já ultrapassaram o limiar crítico de 1.5 °C de aumento da temperatura média em suas superfícies em 2022, valor amplamente referenciado em acordos climáticos internacionais como o Acorde de Paris. Em comparação com 1961, o número de países nessa condição aumentou de forma expressiva, já que neste ano apenas dois países tiveram anomalias maiores que 1.5 °C.

Na comparação entre Brasil e as principais potências economias da América do Sul,Argentina e Comlômbia, nota-se que os países seguem esse padrão global de aquecimento, com as médias brasileira ligeiramente acima da argentina e da comlombiana a partir da década de 90.
refletindo um comportamento semelhante ao das principais potências mundiais.

Quando comparamoss o Brasil com as potênncias globais, China e Estados Unidos, todos os três países demonstram uma tendência clara de aumento nas temperaturas médias da superfície a partir da década de 1980. Destaca-se que a China apresentou o maior crescimento nas médias das temperaturas entre a década de 2010 e o início da década de 2020, enquanto Brasil e Estados Unidos mantiveram-se praticamente estáveis nesse mesmo período.


# 6. Autoavaliação

Seguindo a ideia de desenvlver o MVP sobre dados de mudança climática, foi utilizada uma base de dados proveniente da plataforma FAOSTAT, que disponibiliza estatísticas globais sobre mudanças de temperatura com base em dados da NASA-GISS. Os objetivos propostos neste MVP foram atingidos de maneira satisfatória. Todas as etapas previstas — desde a coleta, preparação, modelagem e análise dos dados — foram realizadas de acordo com o planejamento. .

A modelagem de dados foi desenvolvida em ambiente de nuvem, utilizando a plataforma Databricks Community Edition, com adoção da arquitetura em camadas (medalhão) — bronze, prata e ouro — para garantir organização, rastreabilidade e facilidade de manutenção dos dados ao longo do pipeline.

Foi possível trabalhar com um conjunto de dados que permitiu relacionamentos entre múltiplas tabelas, possibilitando a construção de uma modelagem no esquema estrela, composta por tabelas fato e dimensão, o que favoreceu a prática de joins em consultas em SQL gerando análises mais robustas.

Durante o desenvolvimento, a execução do catálogo de dados e a definição do modelo conceitual exigiram mais esforço do que o inicialmente previsto. Isso se deve à necessidade de delimitar corretamente as entidades e seus relacionamentos, além de estruturar adequadamente a separação entre fatos e dimensões para garantir consistência nas análises.

Como possibilidade de evolução futura do trabalho, sugere-se o uso da versão mais completa do conjunto de dados disponível no site da FAOSTAT, que abrange até o ano de 2024, incluindo médias trimestrais, médias por continente (como Antártida) e os valores de desvio padrão para cada medição. A inclusão dessas informações permitiria enriquecer ainda mais as análises, oferecendo uma visão mais precisa do comportamento das médias das temperaturas na superfície terrestre em diferentes escalas geográficas e temporais.